# Transkript Bot (Colab)
This notebook clones the repo, installs dependencies with uv, and runs the bot.

In [ ]:
# Optional: mount Google Drive for persistent storage
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
import os
from getpass import getpass

os.environ['BOT_TOKEN'] = getpass('BOT_TOKEN: ').strip()
os.environ['HF_TOKEN'] = getpass('HF_TOKEN (optional): ').strip()
os.environ['ROOT_ADMIN_IDS'] = input('ROOT_ADMIN_IDS (comma-separated): ').strip()
# Optional: storage path if using Drive
# os.environ['STORAGE_PATH'] = '/content/drive/MyDrive/transkript/bot.db'
# Optional: force CPU backend
# os.environ['BACKEND_FORCE'] = 'faster'


In [ ]:
import os

REPO_URL = 'https://github.com/USER/REPO.git'
repo_dir = REPO_URL.rstrip('/').split('/')[-1].replace('.git', '')

!pip -q install uv
!git clone $REPO_URL
os.chdir(repo_dir)
!uv venv
!uv sync
# Optional: install WhisperX for GPU diarization
# !uv pip install whisperx


In [ ]:
!uv run python -m transkript_bot.main
